<a href="https://colab.research.google.com/github/jpti89/AB-Wildfires/blob/main/Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install Flask==3.0.0 pyngrok==7.1.2
!pip install dash

  Using cached flask-3.0.0-py3-none-any.whl (99 kB)
  Using cached blinker-1.7.0-py3-none-any.whl (13 kB)
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [21]:
!apt search jupyter
!which jupyter notebook
!ldd keplergl
!pip install jupyter_contrib_nbextensions
!pip install jupyterlab
!pip install keplergl
!jupyter labextension install @jupyter-widgets/jupyterlab-manager keplergl-jupyter
!jupyter contrib nbextension install --user
!jupyter nbextension install --py --sys-prefix keplergl
!jupyter nbextension enable --py --sys-prefix keplergl
!jupyter --version
from IPython.display import HTML

HTML('''
<script src="/nbextensions/keplergl-jupyter/index.js"></script>
''')
from google.colab import output
output.enable_custom_widget_manager()


Sorting... Done
Full Text Search... Done
jupyter/jammy 4.9.1-1 all
  Interactive computing environment (metapackage)

jupyter-client/jammy 7.1.2-1 all
  Jupyter protocol client APIs (tools)

jupyter-console/jammy 6.4.0-3 all
  Jupyter terminal client (script)

jupyter-core/jammy 4.9.1-1 all
  Core common functionality of Jupyter projects (tools)

jupyter-nbconvert/jammy 6.4.0-1 all
  Jupyter notebook conversion (scripts)

jupyter-nbextension-jupyter-js-widgets/jammy 6.0.0-9 all
  Interactive widgets - Jupyter notebook extension

jupyter-nbformat/jammy 5.1.3-1 all
  Jupyter notebook format (tools)

jupyter-notebook/jammy-updates,jammy-security 6.4.8-1ubuntu0.1 all
  Jupyter interactive notebook

jupyter-qtconsole/jammy 5.2.2-1 all
  Jupyter - Qt console (binaries)

jupyter-server/jammy 1.13.1-1 all
  Jupyter protocol server backend (tools)

jupyter-sphinx-theme-common/jammy 0.0.6+ds1-10 all
  Jupyter Sphinx Theme -- common files

jupyter-sphinx-theme-doc/jammy 0.0.6+ds1-10 all
  Jupyter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 43.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyzmq-24.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached jupyter_client-7.4.9-py3-none-any.whl (133 kB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922315 sha256=329b2a5584789f5cd8a26f4a0b79a080651fc2ab0e2af2f1419d36ee8287fcef
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 25.1.2
    Uninstalling pyzmq-25.1.2:
      Successfully uninstalled pyzmq-25.1.2
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter_client 8.6.1
    Uninstalling jupyter_client-8.6.1:
      Successfully uninstalled jupyter_client-8.6

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/usr/local/lib/python3.10/dist-packages/jupyterlab/debuglog.py:56: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/usr/local/lib/python3.10/dist-packages/jupyterlab/debuglog.py:57: UserWarning: ValueError: Please install nodejs >=18.0.0 before continuing. nodejs may be installed using conda or directly from the nodejs website.
  warnings.warn(msg[-1].strip())
/usr/local/lib/python3.10/dist-packages/jupyterlab/debuglog.py:58: UserWarning: See the log file for details: /tmp/jupyterlab-debug-9fo301x0.log
  warnings.warn(f"See the log file for details: {log_path!s}")
[I 00:53:18 InstallContribNbextensionsApp] jupyter contrib nbextension ins

In [22]:
ngrok_key = "2ekHpFtjQMnrMKcsP6FGspAHvSa_3wdrw4jEhBTX5yzRbo5m1"
port = 5000

In [23]:
from pyngrok import ngrok

In [79]:
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://492a-35-245-166-168.ngrok-free.app'

In [80]:
from flask import Flask
from keplergl import KeplerGl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

app = Flask(__name__)

with open('./alberta_788.geojson', 'r') as f:
  boundaries = f.read()

with open('./fp-historical-wildfire-data-2006-2021(2)(1).csv', 'r') as f2:
  historical_fire = f2.read()

df = pd.read_csv("fp-historical-wildfire-data-2006-2021(2)(1).csv")
df

map_1 = KeplerGl()
map_1.add_data(data=boundaries, name='geojson_boundries')
map_1.add_data(data=historical_fire, name='geojson_historical_fire')

@app.route("/")
def hello():
  return render_template('kepler_load.html')

@app.route('/empty_map')
def empty_map():
    return map_1._repr_html_()

@app.route('/chart1')
def chart1():
    fig = go.Figure()
    fig.add_trace(go.Bar(x = np.sort(df.fire_year.unique()),
                    y = df['fire_year'].groupby([df.fire_year]).agg('count'),
                    name='Year of the wildfire',
                    marker_color='rgb(55, 83, 109)'
                    ))

    fig.update_layout(
        title='Wildfires per year from 2006 to 2021',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='Count',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(
            title='Year',
            tickmode='linear'),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1 # gap between bars of the same location coordinate.
    )
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    header="Wildfires per year"
    description = """
    We can visualize a downtrend in number of fires between the years 2006 and 2021.
    """
    return render_template('notdash2.html', graphJSON=graphJSON, header=header,description=description)

@app.route('/chart2')
def chart2():
    fig = px.pie(df['size_class'].groupby([df.size_class]).agg('count'), values='size_class', names=np.sort(df.size_class.unique()))
    fig.update_layout(
        title='Size class of the wildfire')

    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    header="Size of Wildfires"
    description = """
    The size breakdown is as follows:
    A class = 0 to 0.1 ha
    B class > 0.1 ha to 4.0 ha
    C class > 4.0 ha to 40.0 ha
    D class > 40.0 ha to 200 ha
    E class > 200 ha
    """
    return render_template('notdash2.html', graphJSON=graphJSON, header=header,description=description)

@app.route('/chart3')
def chart3():
    fig = go.Figure()
    fig.add_trace(go.Bar(x = np.sort(df.general_cause_desc.unique()),
                y = df['general_cause_desc'].groupby([df.general_cause_desc]).agg('count'),
                name='Year of the wildfire',
                marker_color='rgb(55, 83, 109)'
                ))

    fig.update_layout(
    title='Cause of origin of Wildfires',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
    )
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    header="Cause of Origin"
    description = """
    We can see that the great moyority of Wildfires are caused by Lighting, follow by recreation (Camper, hunter) and residents.
    """
    return render_template('notdash2.html', graphJSON=graphJSON, header=header,description=description)

@app.route('/chart4')
def chart4():
    fig = px.histogram(df, x='relative_humidity')
    fig.update_layout(
    title='Humidity at the start of Wildfire',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
  )
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    header="Humidity"
    description = """
    """
    return render_template('notdash2.html', graphJSON=graphJSON, header=header,description=description)

@app.route('/chart5')
def chart5():
    fig = px.histogram(df, x='temperature')
    fig.update_layout(
    title='Temperature at the start of Wildfire',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
    )
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    header="Temperature"
    description = """
    """
    return render_template('notdash2.html', graphJSON=graphJSON, header=header,description=description)

if __name__ == '__main__':
    app.run(port = port)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
 * Serving Flask app '__main__'
 * Debug mode: off
